<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex09/patrick_ferreira/ex09_patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


#  Exercício: Modelos de Linguagem com Bilhões de parâmetros e Implementação de Algoritmos de Decodificação

Neste exercicio iremos:
1. Avaliar a perplexidade do modelo GPT-2 (1.3B) ou GTP-J (6B) no dataset de testes do IMDB.
2. Ainda usando o GPT-2/J, iremos implementar algorimos de decodificação (sampling, top-k e top-k) e compara-los com os implementados pela função generate da HuggingFace. Esta análise será qualitativa.
3. Implementar uma função similar à torch.multinomial e checar se ela é igual ao do pytorch.

## Instalação/Carregamento dos pacotes

In [2]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 14.6 MB/s 
     |████████████████████████████████| 163 kB 70.1 MB/s 
     |████████████████████████████████| 7.6 MB 51.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 148 kB 14.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62.5 MB 1.2 MB/s 


In [3]:
import collections
import itertools
import functools
import math
import numpy as np
import os
import random
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from typing import List, Type

In [4]:
# Check which GPU we are using
!nvidia-smi

Thu Oct 27 00:54:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


# Carregando o Modelo

In [6]:
# Para desenvolvimento, sugerimos usar o GPT menor, com 1.3B parametros.
model = transformers.GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B", device_map="auto", load_in_8bit=True)
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Se estiver usando o colab pro, conseguimos rodar um GPT-J de 6B de parametros.
# model = transformers.GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16, low_cpu_mem_usage=True)
# tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

# Testando geração com um prompt

In [7]:
prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

verificacao = tokenizer(prompt, return_tensors="pt")

input_ids = tokenizer(prompt, return_tensors="pt").input_ids


In [12]:
prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids.cuda(),
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print("gen_text: ", gen_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


gen_text:  In a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

This discovery, which was made by an international team of researchers led by researcher Dr. Andrew Lacey at the University of Surrey, is the first known record of a herd of unicorns native to South America. While most of the research team have previously studied the phenomenon


# Carregamento do dataset IMDB (conjunto de teste apenas)

Primeiro, fazemos download do dataset:

In [9]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

--2022-10-27 00:57:23--  http://files.fast.ai/data/aclImdb.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.2.19, 104.26.3.19, 172.67.69.159, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.2.19|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/data/aclImdb.tgz [following]
--2022-10-27 00:57:24--  https://files.fast.ai/data/aclImdb.tgz
Connecting to files.fast.ai (files.fast.ai)|104.26.2.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145982645 (139M) [application/x-gtar-compressed]
Saving to: ‘aclImdb.tgz’

aclImdb.tgz         100%[===================>] 139.22M  73.6MB/s    in 1.9s    

2022-10-27 00:57:26 (73.6 MB/s) - ‘aclImdb.tgz’ saved [145982645/145982645]



In [10]:
def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_test = x_test_pos + x_test_neg

print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras validação:')
for x in x_test[:3]:
    print(x[:100])

print('3 últimas amostras validação:')
for x in x_test[-3:]:
    print(x[:100])


25000 amostras de teste.
3 primeiras amostras validação:
I have enjoyed both of the Van Dykes over the years and was glad to watch them again.<br /><br />Jus
I saw this movie at the 2005 Toronto International Film Festival.<br /><br />Based on the novel by J
Richard Schickel's 1991 documentary about Gary Cooper - "Gary Cooper: American Life, American Legend
3 últimas amostras validação:
Adventures in Dinosaur City, though a creative idea, was a nauseatingly atrocious attempt at filmmak
[WARNING: Some spoilers included, though it is a documentary.]<br /><br />I bought this documentary 
There isn't much about "Reckless" that feels right, beginning with the off-putting title (thanks to 


# Classe de dataset

In [ ]:

### Criando classe do dataset

class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int, vocab_size=1000):
        self.context_size = context_size
        try:
            self.x = np.load("x_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            self.y = np.load("y_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            print("Carregando dataset preprocessado")
        except Exception as e:
            # print("Excecao: ", e)
            print("Montando dataset")

            self.x = list()
            self.y = list()

            for text in tqdm(texts):
                tokens_key = tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids
                for i in range(0, len(tokens_key)-context_size-1, context_size):
                    self.x.append(tokens_key[i:i+context_size])
                    self.y.append(tokens_key[i+1:i+context_size+1])

            self.x = np.array(self.x)
            self.y = np.array(self.y)

            np.save("x_" + str(len(texts)) + ".npy", self.x)
            np.save("y_" + str(len(texts)) + ".npy", self.y)

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]).long(), torch.tensor(self.y[idx]).long(), torch.tensor([False] * self.context_size)

# Funções de perplexidade e validação

In [58]:
from torch.cuda.amp import GradScaler, autocast

use_amp = False


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)

def validation_step(input_ids, target_ids, mask):
    model.eval()
    with autocast(enabled=use_amp):
        logits = model(input_ids, attention_mask = mask).logits
        logits = logits.reshape(-1, logits.shape[-1])
        target_ids = target_ids.reshape(-1)
        loss = nn.functional.cross_entropy(logits, target_ids,)
        preds = logits.argmax(dim=1)
    return loss.item()




# Função Multinomial equivalente à do Torch

In [ ]:
# funcao parcialmente inspirada no notebook da Luiza Higino
def my_multinomial(input_tokens):
  
  softmax = nn.Softmax(dim=-1)
  logits = softmax(input_tokens.float())
  random = torch.rand(len(logits))
  aux = random.to(device)*logits.to(device)
  next_token = torch.argmax(aux)

  return next_token

# Dataset de Dataloader de Teste

In [37]:
context_size = 40

test_dataset = MyDataset(texts=x_test, tokenizer=tokenizer, context_size=context_size)
print(f'test examples: {len(test_dataset)}')

test_loader = DataLoader(test_dataset, batch_size=256, num_workers=0)

Montando dataset


  0%|          | 0/200 [00:00<?, ?it/s]

test examples: 1234


# Perplexidade de Teste

In [44]:
with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device), attention_mask.to(device))
        for input, target, attention_mask in tqdm(test_loader)
    ]))

print(f'test perplexity: {test_ppl}')

  0%|          | 0/5 [00:00<?, ?it/s]

test perplexity: 46.97471015107385


# Função de geração de texto 

### Algoritmos Sampling, top-k e top-p 

O usuário pode usar o valor top-k ou top-p nos argumentos da função. Se ambos forem mantidos em None, o algoritmo utilizado é o sampling.

Para do_sampling == False, utiliza-se greedy_decoding

In [67]:
# Codigo da funcao parcialmente inspirado no notebook da Veronica Gesteira
def my_generate(model, tokenized_input, do_sample, temperature = 1.0, top_k = None, top_p = None):
  output = model(tokenized_input.to(device)).logits[0, -1, :]
  if do_sample:
    output = F.softmax(output / temperature, dim = -1) 
    if top_k:
      values, idxs = torch.topk(output, top_k)
      idx = torch.randint(top_k, (1,)).item()
      next_token = idxs[idx].item()
    elif top_p:
      sorted, idxs = torch.sort(output, descending=True)
      sorted_cum = torch.cumsum(sorted, dim=-1)
      last_idx = sorted_cum[sorted_cum < top_p].shape[0] + 1
      idx = my_multinomial(sorted[:last_idx]).item()
      next_token = idxs[idx].item()
    else:
      next_token = my_multinomial(output).item()
  else:
    next_token = torch.argmax(output).item()
  return next_token

# Geração de texto com diferentes estratégias de decoding

### Sampling

In [69]:
prompt = (
    'Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to '
)


input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_length = context_size

for i in range(gen_length):
  next_token = my_generate(model, input_ids, do_sample=True)
  input_ids = torch.cat([input_ids, torch.tensor([[next_token]])], dim=-1)

print(tokenizer.decode(input_ids[0]))

Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to 
skirts training dogs to hunt lions while unfocusedRange
 wielding the power of life and Wenger admittedESS_pulations to coachesMSP_ Countryside at



### Top-K

Usamos k == 8.

In [73]:


input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_length = context_size

for i in range(gen_length):
  next_token = my_generate(model, input_ids, do_sample=True, top_k=8)
  input_ids = torch.cat([input_ids, torch.tensor([[next_token]])], dim=-1)

print(tokenizer.decode(input_ids[0]))

Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to  Mary who had no hope to live in that she was the result from a curse. He is then killed when an army led the army to destroy the



### Top-p

Usamos p == 0.9.

In [71]:


input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_length = context_size

for i in range(gen_length):
  next_token = my_generate(model, input_ids, do_sample=True, top_p=0.9)
  input_ids = torch.cat([input_ids, torch.tensor([[next_token]])], dim=-1)

print(tokenizer.decode(input_ids[0]))

Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to  creatures: the Primitive Monster and his associates, the Dead Toys, and the Blast Furnace and Furnace, all of whom die prematurely in the


### Função generate() da Hugging face

In [77]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids.cuda(),
    do_sample=True,
    temperature=0.9,
    max_length=context_size + len(input_ids),
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to  Frankenstein’s monster. The events happen before and during the time


# Avaliacao das diferentes formas de geraçaõ de texto.


Nota-se que a estratégia Sampling produz textos menos coerentes, como era esperado, por ser uma estratégia mais simples e que não avalia a probabilidade conjunta dos tokens. 

Top-k e Top-p produziram textos diferentes, porém de qualidade aparentemente semelhantes. O final das frases em top-k começa a apresentar certa repetição, enquanto que top-p parece ligeiramente melhor.

A estratégia padrão da Hugging face não parece ser muito melhor que o top-p, eu diria que estão no mesmo nível, já que ambas geram uma história plausível e sem muitas repetições.